In [1]:
# импорт необходимых элементов
import statistics
from scipy.stats import iqr
import json
from datetime import datetime
with open('to_model/parametres.json', 'r') as f:
    param_json = json.load(f)

In [2]:
# импорт модели без PCA

import dill
with open('models/model_dill.pkl', 'rb') as f:
    pipeline = dill.load(f)
pipeline.steps

[('ful', Preprocess_ful()),
 ('transform',
  FeatureUnion(transformer_list=[('dummy_15',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='15',
                                                                 norm='norm')),
                                                  ('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))])),
                                 ('dummy_16',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='16',
                                                                 norm='norm')),
             

In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from sklearn.metrics import f1_score

In [4]:
# загрузка датасета

data = pd.read_csv('git_ignore_files/data_merge_asof.csv')
data.shape

(831653, 259)

In [5]:
# целевая переменная

target = data['target']

In [6]:
# предикт всех данных

%time x = pipeline.predict_proba(data.drop('target', axis=1))[:, 1]

CPU times: user 1min 22s, sys: 26.5 s, total: 1min 48s
Wall time: 1min 41s


In [7]:
# импорт для загрузки параметров

from save_file_param import open_json

param_json = open_json()
param_json

{'names': 'datas',
 'drop_column': [75,
  81,
  85,
  95,
  139,
  203,
  41,
  135,
  10,
  0,
  54,
  8,
  91,
  1,
  3,
  6,
  136,
  78,
  44,
  50,
  39,
  12,
  38,
  13,
  40,
  45,
  101,
  42,
  2,
  11,
  209,
  32,
  4],
 'xgb': 0.27,
 'xgb_pca': 0.45}

In [8]:
# по сохраненному порогу делаем предикт

y_pred = [1 if i > param_json['xgb'] else 0 for i in x]
f1_score(target, y_pred, average='macro')

0.6663123035437689

In [9]:
# проверим на модели с PCA

In [11]:
# загрузка модели с PCA

with open('models/model_pca_dill.pkl', 'rb') as f:
    pipeline_pca = dill.load(f)
pipeline_pca.steps

[('ful', Preprocess_ful()),
 ('transform',
  FeatureUnion(transformer_list=[('dummy_15',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='15',
                                                                 norm='norm')),
                                                  ('one_hot',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse=False))])),
                                 ('dummy_16',
                                  Pipeline(steps=[('column_dummy',
                                                   Preprocessing(aggfunc='median',
                                                                 column='16',
                                                                 norm='norm')),
             

In [12]:
# предикт модели с PCA

y_pred = [1 if i > param_json['xgb_pca'] else 0 for i in pipeline_pca.predict_proba(data)[:, 1]]
f1_score(target, y_pred, average='macro')

0.6647542467610601

# тк модель с PCA показывает результат не хуже, а к метрикам на тренировке гораздо ближе - далее будем испольовать ее